## Logistics Regression 和 DataSet

In [1]:
# read dataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
torch.manual_seed(0)
data = load_breast_cancer()
data.data, data.target

(array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

In [2]:
X = data.data.astype(np.float32)
y = data.target.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train = torch.from_numpy(X_train).type(torch.float32)
X_test = torch.from_numpy(X_test).type(torch.float32)
y_train = torch.from_numpy(y_train).type(torch.float32)
y_test = torch.from_numpy(y_test).type(torch.float32)
X_train

tensor([[-0.1984, -1.2161, -0.2054,  ...,  0.2437,  0.5425,  0.4502],
        [ 1.7041,  1.9777,  1.6390,  ...,  0.5899, -0.8192, -0.0930],
        [ 1.4393,  3.0601,  1.3883,  ...,  0.6841, -0.3435, -0.5067],
        ...,
        [-0.8497, -1.0070, -0.8256,  ..., -0.1512,  0.6701,  0.3163],
        [-0.1065,  1.0347, -0.0393,  ...,  1.2733,  2.6196,  1.2852],
        [-0.7200, -1.1278, -0.7018,  ..., -0.7331, -1.0054, -0.1986]])

In [3]:
# 构造模型

class MyModel(torch.nn.Module):
    def __init__(self, feature):
        super().__init__()
        self.linear = torch.nn.Linear(feature, 1, bias=True)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [4]:
model = MyModel(30)
loss = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

In [5]:
for iter in range(10000):
    y_pred = model(X_train.view(-1, 30))
    l = loss(y_pred.view(-1, 1), y_train.view(-1, 1))
    # print(f'iter: {iter}, loss: {l}')
    l.backward()
    optimizer.step()
    optimizer.zero_grad()
    with torch.no_grad():
        y_test_pred = model(X_test.view(-1, 30))
        y_test_pred = y_test_pred.round().squeeze()
        total_correct = y_test_pred.eq(y_test).sum()
        prec = total_correct.item() / len(y_test)
        print(f'epoch {iter}, loss: {l}, prec: {prec}')

epoch 0, loss: 0.694658100605011, prec: 0.9254385964912281
epoch 1, loss: 0.4307306408882141, prec: 0.9429824561403509
epoch 2, loss: 0.3408064842224121, prec: 0.9517543859649122
epoch 3, loss: 0.29276043176651, prec: 0.9517543859649122
epoch 4, loss: 0.26191771030426025, prec: 0.9517543859649122
epoch 5, loss: 0.24006162583827972, prec: 0.9517543859649122
epoch 6, loss: 0.22358453273773193, prec: 0.9473684210526315
epoch 7, loss: 0.21062318980693817, prec: 0.9517543859649122
epoch 8, loss: 0.20010456442832947, prec: 0.9605263157894737
epoch 9, loss: 0.19136185944080353, prec: 0.9605263157894737
epoch 10, loss: 0.1839558333158493, prec: 0.9605263157894737
epoch 11, loss: 0.1775841861963272, prec: 0.9605263157894737
epoch 12, loss: 0.1720314621925354, prec: 0.9605263157894737
epoch 13, loss: 0.16713926196098328, prec: 0.9605263157894737
epoch 14, loss: 0.16278880834579468, prec: 0.9649122807017544
epoch 15, loss: 0.1588885337114334, prec: 0.9692982456140351
epoch 16, loss: 0.15536725521